In [1]:
from src.ap_core import Ap
import numpy as np
from src.ap_data import load_edges, load_checkins
from tqdm import tqdm

ds_path = "data/loc-gowalla_edges.txt"
chk_path = "data/loc-gowalla_totalCheckins.txt"

In [2]:
def get_intersect(gt, pred):
    gt = set(gt)
    pred = set(pred)
    return len(gt.intersection(pred))


csr_ds = load_edges(ds_path)
checkins_train, checkins_test = load_checkins(chk_path)

In [3]:
ap = Ap()
ap.fit(csr_ds, 1)

  0%|          | 0/1 [00:00<?, ?it/s]/home/jan/PycharmProjects/ML_HOMEWORK4/venv/lib/python3.6/site-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
100%|██████████| 1/1 [00:19<00:00, 19.09s/it]


In [4]:
topn_base = ap.predict(checkins_train)
cluster_prec = 0
base_prec = 0
for user_id in tqdm(np.unique(checkins_test.user_id)):
    gt_locs = checkins_test.loc[checkins_test.user_id == user_id, "location_id"].values
    if user_id in checkins_train.user_id:
        train_locs = ap.predict(checkins_train, user_id)
        cluster_prec += get_intersect(gt_locs, train_locs) / len(gt_locs)
        base_prec += get_intersect(gt_locs, topn_base) / len(gt_locs)
        # test_locs = ap.predict(checkins_test, user_id)



100%|██████████| 31421/31421 [07:38<00:00, 68.56it/s]


In [5]:
print(f"Base AP {base_prec/len(np.unique(checkins_test.user_id))}")
print(f"Cluster AP {cluster_prec/len(np.unique(checkins_test.user_id))}")


Base AP 0.012316603545399573
Cluster AP 0.3302250087521085
